# Setup Notebook - Shared Library Installation

## Purpose
This notebook sets up a shared Python package directory that can be used across all notebooks in this workspace.

## ⚠️ Important Notes
1. **Run this notebook first** before running any other notebooks in the workspace
2. **Re-run after service restarts**: The `/workspace` directory is not persistent across container service restarts
3. Service restarts can occur:
   - During weekend maintenance
   - After idle timeout (default 1 hour)
   - Manual service restarts
4. If other notebooks fail with import errors, re-run this notebook

## Architecture
Instead of each notebook maintaining its own virtual environment, we:
1. Create a shared directory: `/workspace/site-packages_shared`
2. Install all required packages once into this directory
3. Add this directory to Python's path in each notebook

## Benefits
- Install packages once, use across all notebooks
- Faster notebook startup times
- Consistent package versions across notebooks
- Reduced storage usage

## Risks
- No per-notebook environment isolation
- Package version conflicts if different notebooks need different versions
- Must re-run after service restarts (packages are not persistent)


## Step 1: Enable External Access Integration

External access is required to download packages from PyPI and other external sources.


In [ ]:
# This step should be done via SQL or Snowsight UI by an admin
# The external access integration should already be configured for this workspace
# If not, refer to: https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-on-spcs#enable-external-access-integrations-in-snowsight

print("✓ External access should be configured at the workspace/service level")
print("  If package installation fails, check External Access Integration settings")


## Step 2: Create Shared Package Directory


In [ ]:
import os

# Create the shared directory for storing Python packages
shared_dir = "/workspace/site-packages_shared"

!mkdir -p {shared_dir}

print(f"✓ Created shared package directory: {shared_dir}")
print(f"  Directory exists: {os.path.exists(shared_dir)}")


## Step 3: Install All Required Packages

This installs all packages from `requirements.txt` into the shared directory.

**Note**: This may take several minutes depending on the number of packages.


In [ ]:
import time

start_time = time.time()
print("Installing packages to shared directory...")
print("This may take 5-10 minutes...\n")

# Install all packages from requirements.txt to the shared directory
!pip install --target /workspace/site-packages_shared -r requirements.txt

elapsed_time = time.time() - start_time
print(f"\n✓ Package installation complete!")
print(f"  Time elapsed: {elapsed_time:.1f} seconds ({elapsed_time/60:.1f} minutes)")


## Step 4: Verify Installation


In [ ]:
import sys
import os

SITE_SHARED = "/workspace/site-packages_shared"

# Add to Python path if not already there
if SITE_SHARED not in sys.path:
    sys.path.append(SITE_SHARED)

print("Verifying installed packages:\n")

# List of key packages from requirements.txt
test_packages = [
    "rioxarray",
    "pystac_client",
    "planetary_computer",
    "odc",
    "shapely",
    "rasterio",
    "pandas",
    "xarray",
    "matplotlib",
    "geopandas",
    "seaborn",
    "netCDF4",
    "adlfs",
    "zarr",
    "dask"
]

success_count = 0
fail_count = 0

for package in test_packages:
    try:
        __import__(package)
        print(f"✓ {package}")
        success_count += 1
    except ImportError as e:
        print(f"✗ {package} - FAILED: {e}")
        fail_count += 1

print(f"\n{'='*50}")
print(f"Verification Results:")
print(f"  Success: {success_count}/{len(test_packages)}")
print(f"  Failed: {fail_count}/{len(test_packages)}")

if fail_count == 0:
    print(f"\n🎉 All packages installed successfully!")
    print(f"   You can now run any notebook in this workspace.")
else:
    print(f"\n⚠️  Some packages failed to install.")
    print(f"   Check the installation output above for errors.")


In [ ]:
# Show all installed packages in the shared directory
print("Packages installed in shared directory:\n")
!ls -la /workspace/site-packages_shared | head -30


## Setup Complete! ✓

### Next Steps:
1. All other notebooks in this workspace should include this code at the top:

```python
import sys

SITE_SHARED = "/workspace/site-packages_shared"

if SITE_SHARED not in sys.path:
    sys.path.append(SITE_SHARED)
```

2. Run your notebooks normally - no need to install packages again!

### Troubleshooting:
- **Import errors in notebooks**: Re-run this setup notebook
- **Service restarted**: Re-run this setup notebook
- **Package conflicts**: May need to adjust requirements.txt versions
- **External access errors**: Check workspace External Access Integration settings
